In [1]:
# install missing package
%pip install telethon
%pip install python-dotenv

import os
import json
import asyncio
from datetime import datetime

from telethon import TelegramClient
from dotenv import load_dotenv

# load environment variables from .env file
load_dotenv()


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


True

In [2]:
# Load secrets from .env
load_dotenv()

API_ID = int(os.getenv("TELEGRAM_API_ID"))
API_HASH = os.getenv("TELEGRAM_API_HASH")

API_ID, bool(API_HASH)


(36146350, True)

In [3]:
CHANNEL_CONFIG = {
    "lobelia4cosmetics": 150,
    "tikvahpharma": 130,
    "chemed123":120
}

BASE_DATA_PATH = "data/raw/telegram_messages"



In [4]:
def ensure_dir(path: str):
    if not os.path.exists(path):
        os.makedirs(path)


In [ ]:
async def scrape_telegram_channels():
    client = TelegramClient("notebook_session", API_ID, API_HASH)
    await client.start()

    today = datetime.utcnow().strftime("%Y-%m-%d")

    for channel, limit in CHANNEL_CONFIG.items():
        print(f"\n🌿 Scraping {channel} (limit={limit})")

        messages_data = []
        scraped_count = 0

        async for msg in client.iter_messages(channel, limit=limit):
            scraped_count += 1

            progress = (scraped_count / limit) * 100
            print(f"  → Progress: {progress:.0f}% ({scraped_count}/{limit})", end="\r")

            messages_data.append({
                "message_id": msg.id,
                "channel_name": channel,
                "date": msg.date.isoformat() if msg.date else None,
                "text": msg.text,
                "views": msg.views,
                "forwards": msg.forwards,
                "has_media": bool(msg.media)
            })

        print(f"\n✅ Finished {channel}: {scraped_count} messages scraped")

        channel_dir = f"{BASE_DATA_PATH}/{today}"
        ensure_dir(channel_dir)

        file_path = f"{channel_dir}/{channel}.json"
        with open(file_path, "w", encoding="utf-8") as f:
            json.dump(messages_data, f, indent=2, ensure_ascii=False, default=str)

    await client.disconnect()




In [ ]:
await scrape_telegram_channels()